In [ ]:
!rm -rf qber-forecasting
!rm -rf deep_qber

In [ ]:
!git clone https://github.com/rmnigm/qber-forecasting.git
!pip install wandb
!pip install pytorch_lightning torchmetrics

In [ ]:
!cp -r qber-forecasting/deep_qber deep_qber

In [ ]:
import os
import random
import sys

import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import mean_squared_error, mean_absolute_percentage_error 

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

In [ ]:
from deep_qber import seed_everything, setup_dataset
from deep_qber import TorchTSDataset, ModelInterfaceTS, ModuleTS

In [ ]:
# optional
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class TorchTSDataset(Dataset):
    def __init__(self,
                 dataset,
                 target_index=0,
                 look_back=1,
                 device='cpu'):
        length = dataset.shape[0] - look_back - 1
        width = dataset.shape[1]
        mask = np.array([i != target_index for i in range(width)])
        x_current = np.empty((length, 1, width - 1))
        x, y = np.empty((length, look_back, width)), np.empty((length, 1))
        for i in range(length):
            x[i] = dataset[i:(i + look_back), :]
            x_current[i] = dataset[i + look_back, mask]
            y[i] = dataset[i + look_back, target_index]
        self.X = torch.tensor(x).float().to(device)
        self.y = torch.tensor(y).float().to(device)
        self.X_current = torch.tensor(x_current).float().to(device)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return (self.X[idx], self.X_current[idx]), self.y[idx]


def setup_dataset(dataset,
                  look_back: int = 5,
                  train_size: float = 0.8,
                  scaler=None,
                  batch_size: int = 64,
                  shuffle: bool = False,
                  device: str = 'cpu'):
    train_size = int(len(dataset) * train_size)
    test_size = len(dataset) - train_size
    data_train, data_test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
    print("Training set size = {}, testing set size = {}".format(train_size, test_size))

    if scaler is not None:
        scaler.fit(data_train)
        data_train = scaler.transform(data_train)
        data_test = scaler.transform(data_test)

    train_set = TorchTSDataset(data_train,
                               target_index=0,
                               look_back=look_back,
                               device=device)
    train_loader = DataLoader(train_set,
                              batch_size=batch_size,
                              shuffle=shuffle)
    test_set = TorchTSDataset(data_test,
                              target_index=0,
                              look_back=look_back,
                               device=device)
    test_loader = DataLoader(test_set,
                             batch_size=batch_size,
                             shuffle=shuffle)
    return train_loader, test_loader

In [ ]:
class ModuleTS(pl.LightningModule):
    def __init__(self, model, loss, lr=1e-5):
        super().__init__()
        self.model = model
        self.loss = loss
        self.lr = lr
        self.loss_multiplier = 1e4
        self.save_hyperparameters(ignore=['model'])

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        data, target = train_batch
        predictions = self.forward(data)
        loss = self.loss_multiplier * self.loss(predictions, target)
        self.log("Train Loss", loss, prog_bar=True)
        metrics = self.model.get_metrics(predictions, target)
        self.log("Train MSE", metrics["MSE"], prog_bar=True)
        self.log("Train MAPE", metrics["MAPE"], prog_bar=True)
        return loss

    def validation_step(self, val_batch, batch_idx):
        data, target = val_batch
        preds = self.forward(data)
        loss = self.loss_multiplier * self.loss(preds, target)
        metrics = self.model.get_metrics(preds, target)
        self.log("Validation Loss", loss, prog_bar=True)
        self.log("Validation MSE", metrics["MSE"], prog_bar=True)
        self.log("Validation MAPE", metrics["MAPE"], prog_bar=True)

In [116]:
pulses_stats_file_path = "/content/qber-forecasting/datasets/data.csv"
dataframe = pd.read_csv(pulses_stats_file_path,
                        usecols=range(1, 8),
                        engine='python',
                        )
dataframe = dataframe[:100000]
dataset = dataframe.values.astype('float32')

In [113]:
import statistics
from collections import deque

class OutlierDetector:
    def __init__(self, mode='median', window_size=10, alpha=3):
        self.mode = 'median'
        self.window_size = window_size
        self.alpha = alpha

    def fit(self, ts):
        anomalies = []
        for i in range(len(ts) - self.window_size):
            left_border = max(0, i - self.window_size)
            right_border = i
            med = np.median(ts[left_border:right_border])
            std = np.std(ts[left_border:right_border])
            diff = np.abs(ts[i] - med)
            if diff > std * self.alpha:
                anomalies.append(i)
        return anomalies
    
    def fit_transform(self, ts):
        new_ts = list(ts[:self.window_size].copy())
        window = deque(ts[:self.window_size])
        for item in tqdm(ts[self.window_size:]):
            med = statistics.median(window)
            std = statistics.stdev(window)
            diff = np.abs(item - med)
            if diff > std * self.alpha:
                new_ts.append(med)
            else:
                new_ts.append(item)
                window.append(item)
                window.popleft()
        return new_ts

In [117]:
outl = OutlierDetector(window_size=100, alpha=3)
no_outliers = outl.fit_transform(dataframe['e_mu_current'].values)

100%|██████████| 99900/99900 [00:32<00:00, 3094.03it/s]


In [126]:
dataframe['e_mu_current'] = no_outliers
transformed_dataset = dataframe.values.astype('float32')

In [118]:
config = {
    "learning_rate": 1e-5,
    "look_back": 50,
    "input_size": 7,
    "output_size": 1,
    "hidden_size": 512,
    "batch_size": 64,
    "epochs": 30,
    "loss": "MSE",
    "scaler": None,
    "model": "autoregressive_with_future_knowledge"
}
train_size = 0.8
loss = nn.MSELoss()
scaler = None

In [120]:
seed_everything(42)
train_loader, test_loader = setup_dataset(dataset,
                                          config["look_back"],
                                          train_size,
                                          config["scaler"],
                                          config["batch_size"],
                                          device=device)

input_size = config["input_size"]
look_back = config["look_back"]
hidden_size = config["hidden_size"]
output_size = config["output_size"]

Training set size = 80000, testing set size = 20000


In [121]:
class Extractor(nn.Module):
    def __init__(self, look_back, output_size, hidden_size):
        super().__init__() 
        self.input_size = look_back
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.autoregressive_dense = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
        )
        self.classifier = nn.Sequential(
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.output_size)
        )
        
    def forward(self, data):
        x, x_current = data
        autoregressive_features = self.autoregressive_dense(x[:, :, 0])
        return self.classifier(autoregressive_features)

In [ ]:
class ExtractorExod(nn.Module):
    def __init__(self, look_back, output_size, hidden_size):
        super().__init__() 
        self.input_size = look_back
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.autoregressive_dense = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
        )
        self.dense_exod = nn.Sequential(
            nn.Linear(6, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size, 64),
            )
        self.classifier = nn.Sequential(
            nn.LeakyReLU(),
            nn.Linear(self.hidden_size + 64, self.output_size)
        )
        
    def forward(self, data):
        x, x_current = data
        autoregressive_features = self.autoregressive_dense(x[:, :, 0])
        exod_features = self.dense_exod(x_current)[:, -1, :]
        return self.classifier(torch.cat((autoregressive_features, exod_features), 1))

In [ ]:
class ExtractorLSTM(nn.Module):
    def __init__(self, input_size, output_size, hid_size=128):
        """
        Базовая модель encoder-decoder архитектуры
        """
        super().__init__() 
        self.input_size = input_size
        self.output_size = output_size
        self.hid_size = hid_size
        self.lstm = nn.LSTM(input_size,
                            hid_size,
                            batch_first=True
                            )
        self.dense = nn.Sequential(
            nn.Linear(input_size - 1, hid_size),
            nn.LeakyReLU(),
            nn.Linear(hid_size, hid_size),
        )
        self.regressor = nn.Linear(2 * hid_size, output_size)
        
    def forward(self, data):
        x, x_current = data
        x, _ = self.lstm(x)
        past_features = x[:, -1, :]
        current_features = self.dense(x_current)[:, -1, :]
        features = torch.cat((past_features, current_features), 1)
        return self.regressor(features)

In [122]:
model = Extractor(look_back=config["look_back"],
                  hidden_size=config["hidden_size"],
                  output_size=config["output_size"]
                  )

In [ ]:
model = ExtractorLSTM(input_size=config["input_size"],
                      hid_size=config["hidden_size"],
                      output_size=config["output_size"]
                      )

In [123]:
metrics = {
    "MSE": mean_squared_error,
    "MAPE": mean_absolute_percentage_error
    }

In [124]:
def run_experiment(train_loader, test_loader, model, loss, config, name):
    with wandb.init(project="qber-forecasting",
                    entity="rmnigm",
                    settings=wandb.Settings(start_method="thread"),
                    config=config,
                    name=name,
                    ) as run:
        wandb_logger = WandbLogger(log_model='all')
        checkpoint_callback = ModelCheckpoint(monitor="Validation MAPE", mode="min")

        epochs = config["epochs"]

        model_interface = ModelInterfaceTS(model)
        module = ModuleTS(model_interface, loss, lr=config["learning_rate"])

        trainer = pl.Trainer(logger=wandb_logger,
                            callbacks=[checkpoint_callback],
                            accelerator="gpu",
                            max_epochs=epochs,
                            )
        
        trainer.fit(module, train_loader, test_loader)

        run.finish()

In [125]:
name = config["model"]


run_experiment(train_loader, test_loader, model, loss, config, name)

wandb: Currently logged in as: rmnigm. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:396: UserWarning:

There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning:

Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.mo

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


Train Loss,▃▂▆▂▂▂▂▂▂▁▂▂▁▁▂▂▁█▇▁▁▁▁▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁█
Train MAPE,▄▃█▃▂▃▃▃▂▂▂▂▁▂▃▂▂▄▇▁▁▂▂▄▁▂▂▂▁▁▁▁▂▂▁▂▁▁▂█
Train MSE,▃▂▆▂▂▂▂▂▂▁▂▂▁▁▂▂▁█▇▁▁▁▁▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁█
Validation Loss,█▇▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation MAPE,██▇▇▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation MSE,█▇▅▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
Train Loss,0.01402
Train MAPE,0.09098
Train MSE,0.0


In [133]:
loader, _ = setup_dataset(transformed_dataset[46000:48000],
                          look_back=config["look_back"],
                          train_size=0.75,
                          shuffle=False,
                          batch_size=config["batch_size"],
                          device=device)

model.eval().to(device)

predictions = []
targets = []
with torch.no_grad():
    for batch in loader:
        data, target = batch
        targets.append(target)
        predictions.append(model(data))

Training set size = 1500, testing set size = 500


In [163]:
flat_predictions = []
flat_transformed = []
flat_actual = list(dataset[46000:47500][50:-1, 0])

for batch in predictions:
    flat_predictions += list(batch.detach().to('cpu').numpy().flatten())

for batch in targets:
    flat_transformed += list(batch.detach().to('cpu').numpy().flatten())

model_quality_df = pd.DataFrame({'E_mu (no outliers)': flat_transformed,
                                 'E_mu (actual)': flat_actual,
                                 'predicted value': flat_predictions
                                 }).reset_index()

model_quality_df.to_csv(config['model'] + '.csv')

In [164]:
cols = ['E_mu (actual)', 'predicted value', 'E_mu (no outliers)']
fig = px.line(model_quality_df, x='index', y=cols)
fig.show()